# Solving Hangman

## What is Hangman?

`text`

## Project Goals

* Explore several player strategies
* Explore several adversary strategies
* Compare different players vs. different adversaries
* Attempt to reach an equillibrium for the player on one side, and adversary on the other side

## Proposed Code structure

The code is subdivided as follows:

* `Hangman.ipynb` - this notebook, containing discussion and experiments.
* `game.py`, implementing the `Game` class, which stores game parameters and rules.
* `player.py`, implementing various player strategies.
* `adversary.py`, implementing various adversary strategies.
* `functions.py`, implementing universal functions used throughout this project.

## Word list

Both the player and the adversary use the same known word list. For this work, we use a list included with `NLTK`, converted to lowercase to improve consistency of player and adversary strategies.

In [1]:
from nltk.corpus import words

wl = [w.lower() for w in words.words()]

# Short test word list for more locals tests
# wl = ['robby', 'dobby', 'bobby', 'dobbs']

## Player Strategies: 



In [6]:
import game
import player
import adversary

### Strategy 0: 'Drunken' Random Player

In [7]:
g = game.Game(wl, print_progress=True)
w = adversary.adv0(g)
g.set_word(w)
player.play0(g)

print(g.word_mask)
g.print_word()

Incorrect. Attempts left: 7
Incorrect. Attempts left: 6
Incorrect. Attempts left: 5
Incorrect. Attempts left: 4
Incorrect. Attempts left: 3
Incorrect. Attempts left: 2
Incorrect. Attempts left: 1
Incorrect. Attempts left: 0
brass
*****


### Strategy 1: Using fixed letter frequency in all words

In [8]:
g = game.Game(wl, print_progress=True)
w = adversary.adv0(g)
g.set_word(w)
player.play1(g)

print(g.word_mask)
g.print_word()

Correct!  *i*****
Correct!  *i*i***
Correct!  *i*i*e*
Incorrect. Attempts left: 7
Incorrect. Attempts left: 6
Incorrect. Attempts left: 5
Correct!  *i*i*en
Correct!  *iti*en
Incorrect. Attempts left: 4
Incorrect. Attempts left: 3
Correct!  citi*en
Incorrect. Attempts left: 2
Incorrect. Attempts left: 1
Incorrect. Attempts left: 0
citizen
citi*en


### Strategy 2: Using frequencies for words of known length

In [9]:
g = game.Game(wl, print_progress=True)
w = adversary.adv0(g)
g.set_word(w)
player.play2(g)

print(g.word_mask)
g.print_word()

Correct!  *******i**
Correct!  **e****i**
Correct!  **e**e*i**
Incorrect. Attempts left: 7
Incorrect. Attempts left: 6
Incorrect. Attempts left: 5
Incorrect. Attempts left: 4
Correct!  **e**e*it*
Incorrect. Attempts left: 3
Correct!  s*e**e*it*
Incorrect. Attempts left: 2
Incorrect. Attempts left: 1
Incorrect. Attempts left: 0
skedgewith
s*e**e*it*


### Strategy 3: Word list bisect-based player using only correct letters

In [11]:
g = game.Game(wl, print_progress=True)
w = adversary.adv0(g)
g.set_word(w)
player.play3(g)

print(g.word_mask)
g.print_word()

Correct!  ****e********
Correct!  ****e*i******
Correct!  ****e*i***i**
Correct!  ****e*i**ti**
Correct!  ****e*i**ti*n
Correct!  ****e*i**tion
Correct!  **a*e*i**tion
Correct!  **a*e*i*ation
Correct!  **a*e*ization
Incorrect. Attempts left: 7
Correct!  *la*e*ization
Correct!  *la*elization
Correct!  *lavelization
Correct!  clavelization
clavelization
clavelization
